Imports


In [39]:
import numpy as np
import pandas as pd
import os
import json
from collections import Counter
import torch
import nltk
nltk.download('all')
import re
import onnx
import onnxruntime

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\james\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\james\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\james\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\james\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\james\AppData\Roaming\nltk_data...
[nltk_data]    | 

Gather Data

In [40]:
#Combine json files
dataDir = '../WebScraper/data/'
files = [os.path.join(dataDir, f) for f in os.listdir(dataDir) if f.endswith('.json')]

result = list()
for f1 in files:
    with open(f1, 'r') as infile:
        result.extend(json.load(infile))

with open('data.json', 'w') as output_file:
    json.dump(result, output_file)

In [41]:
#Read json file into pandas data frame
data = pd.read_json('data.json', encoding='utf-8-sig')


Remove Unusable Posts

In [42]:
#Remove empty texts
numtexts = data.shape[0]
data = data.dropna()
print ("Removed " + str(numtexts - data.shape[0]) + " empty posts.")

#Remove duplicates
numPosts = data.shape[0]
data = data.drop_duplicates()
print ("Found " + str(data.shape[0]) + " unique posts from " + str(numPosts) + " posts scraped.")

#Remove posts that threw errors when collecting
numPosts = data.shape[0]
data = data[data['valid'] == 'true']
print ("Removed " + str(numPosts - data.shape[0]) + " posts with errors.")

#Remove 'valid' column as it will not be used any more
data = data.drop(['valid'], axis=1)

#Merge the title and body into one column
data['body'] = data['title'] + ' ' + data['body']
data = data.drop(['title'], axis=1)


Found 9976 unique posts from 17282 posts scraped.
Removed 94 empty posts.
Removed 0 posts with errors.


In [43]:
#Remove posts in non-decision categories
data = data[data['decision'] != 'UPDATE']
data = data[data['decision'] != 'Update']
data = data[data['decision'] != 'YTA reddit']
data = data[data['decision'] != 'Open Forum']
data = data[data['decision'] != 'POO Mode Activated ']
data = data[data['decision'] != 'META']
data = data[data['decision'] != 'Not enough info POO Mode']
data = data[data['decision'] != 'TL;DR']
data = data[data['decision'] != 'Not enough info']
data = data[data['decision'] != 'Charitable META']

print ("Removed non-decision categories:\n")
decisions = data['decision']
print (Counter(decisions))
print ()

Removed non-decision categories:

Counter({'Not the A-hole': 6385, 'Asshole': 2267, 'No A-holes here': 380, 'Everyone Sucks': 322, 'Not the A-hole POO Mode': 46, 'Asshole POO Mode': 31, 'Everyone Sucks POO Mode': 3, 'No A-holes here POO Mode': 2})



In [44]:
#Simplify categories into a binary
data['decision'] = data['decision'].replace('Not the A-hole POO Mode', 'Not the A-hole')
data['decision'] = data['decision'].replace('Everyone Sucks', 'Asshole')
data['decision'] = data['decision'].replace('No A-holes here', 'Not the A-hole')
data['decision'] = data['decision'].replace('Asshole POO Mode', 'Asshole')
data['decision'] = data['decision'].replace('Everyone Sucks POO Mode', 'Asshole')
data['decision'] = data['decision'].replace('No A-holes here POO Mode', 'Not the A-hole')

#Set labels to 0 and 1
data['decision'] = data['decision'].replace('Not the A-hole', 0)
data['decision'] = data['decision'].replace('Asshole', 1)
print (Counter(data['decision']))

Counter({0: 6813, 1: 2623})


C:\Users\james\AppData\Local\Temp\ipykernel_20044\2923309056.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['decision'] = data['decision'].replace('Asshole', 1)


Build Vocab

In [45]:
#Clean post with stemming and stopwords
#WARNING: takes ~5 minutes
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
count = 0
for post in data['body']:
    count += 1
    print (count)
    post = re.sub('[^a-zA-Z]', ' ', post) #remove non-alphabetic characters
    post = post.lower() 
    post = post.split()
    post = [ps.stem(word) for word in post if not word in set(stopwords.words('english'))] #remove stopwords
    post = ' '.join(post)
    corpus.append(post)

print (corpus[150])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
#Vectorize
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

# Reduce each post to 1000 words, 
# remove words that appear in less than 5 posts, 
# remove words that appear in more than 70% of the posts
vectorizer = TfidfVectorizer(max_features=1500, min_df=3, max_df=0.8)
fitted = vectorizer.fit_transform(corpus)
X = fitted.toarray()
Y = data['decision'].values
X[0]

array([0.        , 0.12338861, 0.        , ..., 0.        , 0.        ,
       0.        ])

In [ ]:
#Save vectorizer as pickle
#Use this if re-releasing
import pickle
pickle.dump(vectorizer, open('vectorizer2.pkl', 'wb'))

In [ ]:
#TEST: load vectorizer from pickle
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = pickle.load(open('vectorizer2.pkl', 'rb'))

In [ ]:
#Vectorizer Test Post
post = "It was the day after graduation when my mom and her boyfriend told me that now that i was an adult, I had to move out and find a place of my own. Although it was unexpected,my mom and I's relationship changed drastically when her new boyfriend moved in. I tried to make a good impression because I wanted her to be happy but the guy just seemed to have an issue with me. He convinced my mom to start charging me rent because I had a part time job while still in school even though he was jobless and living off my mom.He would go in my room, go through my stuff and use some of my things without my permission. He even broke my laptop after using it without asking but my mom saw nothing wrong with his behaviour and always took his side. She would yell at me whenever I complained about how I didn't like him invading my privacy. He seemed very pleased with himself when my mom kicked me out.I didn't have enough to get a place so I survived by sleeping on friends couches as much as I could until they got tired of me and I had to sleep outside at times. Thankfully I was able to get a second job through someone I knew and I was able to save up and I just moved into a new place at the beginning of this month after months of not having anywhere to call home.A few days ago my mom's boyfriend reached out to me. Apparently my mom had a stroke and he tried guilting me into moving back to help her out because he got a job opportunity and has to move. I declined and told him that unfortunately I'm too busy trying to be an adult, it doesn't look like I can take time off that schedule to do that then hung up."
post = re.sub('[^a-zA-Z]', ' ', post) #remove non-alphabetic characters
post = post.lower() 
post = post.split()
post = [ps.stem(word) for word in post if not word in set(stopwords.words('english'))] #remove stopwords
post = ' '.join(post)

#Vectorize
post = vectorizer.transform([post]).toarray()
post

array([[0.        , 0.13525831, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
#Split data into training and testing
from sklearn.model_selection import train_test_split
XTrain, XTest, YTrain, YTest = train_test_split(X, Y, test_size = 0.3, random_state=0)

XTrain_ = torch.from_numpy(XTrain).float()
XTest_ = torch.from_numpy(XTest).float()
YTrain_ = torch.from_numpy(YTrain.astype(np.double)).float()
YTest_ = torch.from_numpy(YTest.astype(np.double)).float()

Build Classifier

In [ ]:
#Define Model parameters
INPUT_SIZE = XTrain_.shape[1] #Number of words considered
OUTPUT_SIZE = 2 #Number of labels
HIDDEN_SIZE = 2000 #This can be changed

In [ ]:
#Create Model
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = torch.nn.Linear(INPUT_SIZE, HIDDEN_SIZE) 
        self.fc2 = torch.nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE)
        self.fc3 = torch.nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE)
        self.fc4 = torch.nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE)  
    
    def forward(self, X):
        X = torch.relu((self.fc1(X)))
        X = torch.relu((self.fc2(X)))
        X = torch.relu((self.fc3(X)))
        X = self.fc4(X)
        return torch.nn.functional.log_softmax(X, dim=1)

model = Model()

In [ ]:
#Create Optimizer
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.001)
lossFunction = torch.nn.CrossEntropyLoss()

Train

In [ ]:
#Train Model
epochs = 150
for epoch in range(epochs):
    optimizer.zero_grad()
    prediction = model(XTrain_)
    loss = lossFunction(prediction, YTrain_.long())
    loss.backward()
    optimizer.step()
    print ("Epoch: " + str(epoch) + " Loss: " + str(loss.item()))

Epoch: 0 Loss: 0.6890143156051636
Epoch: 1 Loss: 0.6178194880485535
Epoch: 2 Loss: 0.5981003046035767
Epoch: 3 Loss: 0.5816494822502136
Epoch: 4 Loss: 0.5783277750015259
Epoch: 5 Loss: 0.575336217880249
Epoch: 6 Loss: 0.5654364228248596
Epoch: 7 Loss: 0.5586879849433899
Epoch: 8 Loss: 0.5534140467643738
Epoch: 9 Loss: 0.5403730869293213
Epoch: 10 Loss: 0.5298528671264648
Epoch: 11 Loss: 0.5180609822273254
Epoch: 12 Loss: 0.5024842619895935
Epoch: 13 Loss: 0.4877789318561554
Epoch: 14 Loss: 0.46813374757766724
Epoch: 15 Loss: 0.4484720230102539
Epoch: 16 Loss: 0.4251156449317932
Epoch: 17 Loss: 0.39876291155815125
Epoch: 18 Loss: 0.36889079213142395
Epoch: 19 Loss: 0.33901137113571167
Epoch: 20 Loss: 0.30689117312431335
Epoch: 21 Loss: 0.27585089206695557
Epoch: 22 Loss: 0.24891559779644012
Epoch: 23 Loss: 0.22961682081222534
Epoch: 24 Loss: 0.21908852458000183
Epoch: 25 Loss: 0.2028251588344574
Epoch: 26 Loss: 0.2042447030544281
Epoch: 27 Loss: 0.1912168562412262
Epoch: 28 Loss: 0.1908

Test

In [ ]:
#Test model on testing posts
results = model(XTest_)
correctYTA = 0
correctNTA = 0
incorrectYTA = 0
incorrectNTA = 0

for i, result in enumerate(results):
    if result.argmax() == YTest_[i]:
        #print ("Correct: " + str(result.argmax()) + " " + str(YTest_[i]))
        if (YTest_[i] == 1):
            correctYTA += 1
        else:
            correctNTA += 1
    else:
        #print ("Incorrect: " + str(result.argmax()) + " " + str(YTest_[i]))
        if (YTest_[i] == 1):
            incorrectYTA += 1
        else:
            incorrectNTA += 1


print ("YTA Correct: " + str(correctYTA))
print ("YTA Incorrect: " + str(incorrectYTA))
print ("YTA Accuracy: " + str(correctYTA / (correctYTA + incorrectYTA)))

print ("\nNTA Correct: " + str(correctNTA))
print ("NTA Incorrect: " + str(incorrectNTA))
print ("NTA Accuracy: " + str(correctNTA / (correctNTA + incorrectNTA)))

print ("\nOverall Accuracy: " + str((correctYTA + correctNTA) / len(YTest_)))

print ("\nConfusion Matrix: \n" + str(sklearn.metrics.confusion_matrix(YTest_, results.argmax(1))))

YTA Correct: 204
YTA Incorrect: 619
YTA Accuracy: 0.24787363304981774

NTA Correct: 1691
NTA Incorrect: 317
NTA Accuracy: 0.8421314741035857

Overall Accuracy: 0.669374779229954

Confusion Matrix: 
[[1691  317]
 [ 619  204]]


Export

In [ ]:
#Save trained model as a .pt file
scripted_model = torch.jit.script(model)
scripted_model.save('aholeClassifier.pt')